# This is our Jupyter notebook

## Crimes worth 2 points
1. Criminal Homicide
2. Forcible Rape
3. Robbery
4. Aggravated Assault
5. Burglary 
6. Larceny-theft (except motor vehicle theft)
7. Motor Vehicle Theft
8. Arson
----------------------------------------
## Crimes worth one point
9. Other Assaults
10. Forgery and Counterfeiting
11. Fraud
12. Embezzlement
13. Stolen Property:  Buying, Receiving, Possessing
14. Vandalism
15. Weapons:  Carrying, Possessing, etc.
16. Prostitution and Commercialized Vice
17. Sex Offenses
18. Drug Abuse Violations
19. Gambling
20. Offenses Against the Family and Children
21. Driving Under the Influence
22. Liquor Laws
23. Drunkenness
24. Disorderly Conduct
25. Vagrancy
26. All Other Offenses
27. Suspicion
28. Curfew and Loitering Laws—(Persons under 18)
29. Runaways—(Persons under 18)

In [41]:
import pandas as pd

crimes = pd.read_csv("crime.csv")
census = pd.read_csv("population-density.csv")

scores = { i : 0 for i in census["Neighborhood"].tolist() }


for i in range(len(crimes)):
    try:
        if crimes["HIERARCHY"][i] < 9:
            scores[crimes["INCIDENTNEIGHBORHOOD"][i]] += 2
        else:
            scores[crimes["INCIDENTNEIGHBORHOOD"][i]] += 1
    except:
        pass

big_hoods = scores.copy()
    
for i in range(len(census)):
    scores[census["Neighborhood"][i]] = scores[census["Neighborhood"][i]] / int(census["Pop. 2010"][i])
    big_hoods[census["Neighborhood"][i]] = big_hoods[census["Neighborhood"][i]] / int(census["Pop. 2010"][i])
    
    if int(census["Pop. 2010"][i]) < 100:
        big_hoods.pop(census["Neighborhood"][i])
    
scores = pd.DataFrame.from_dict(scores, orient='index')   
big_hoods = pd.DataFrame.from_dict(big_hoods, orient='index')   

In [34]:
for i in range(len(crimes)):

    if crimes["INCIDENTNEIGHBORHOOD"][i] == "Chateau":
        print(crimes["INCIDENTHIERARCHYDESC"][i])

nan
nan
nan
nan
CRIMINAL MISCHIEF (AUTO)
nan
DRIVING UNDER INFLUENCE
THEFT FROM AUTO
DRIVING UNDER INFLUENCE
nan
nan
nan
nan
CONVICT NOT TO OWN/MAN/USE/SELL FIREARMS
nan
nan
nan
HARRASSMENT BY COMUN/ADDRESS
PUBLIC DRUNKENESS
THEFT FROM AUTO
PROPULSION OF MISSILES INTO OCC VEH/RDWY/NO INJ
nan
CREDIT CARDS
nan
CRIM MISCHIEF (GRAFFITI)
nan
TERRORISTIC THREATS
nan
nan
COMPUTER TRESSPASS - ALTER/ERASE DATA
nan
THEFT FROM BUILDING
HARRASSMENT-OTHER
nan
nan
nan
CRIM MISCHIEF (GRAFFITI)
nan
THEFT FROM AUTO
nan
nan
nan
THEFT FROM AUTO
nan
nan
nan
THEFT/ALL OTHERS
HARRASSMENT/THREAT/ATTEMPT/PHY
nan
OTHER DRUG/NON-NARC-POSSESS
SIMPLE ASSAULT/INJURY
MARIJUANA-POSSESS
nan
nan
nan
nan
nan
INVASION PRIVACY - VICTIM FULL OR PART NUDE
THEFT FROM AUTO
nan
nan
nan
AGG ASSAULT/OTHER/INJURY
THEFT/ALL OTHERS
OBST ADM LAW /OTHR GOVT FUNCTN
OBST ADM LAW /OTHR GOVT FUNCTN
nan
CRUELTY TO ANIMALS
nan
nan
nan
nan
SIMPLE ASSAULT/INJURY
THEFT/PROP-LOST/MISLAID/DEL
SIMPLE ASSAULT/INJURY
CRIMINAL MISCHIEF (AUTO)
BAD 